- Reference
  - https://github.com/microsoft/LightGBM/blob/master/examples/python-guide/simple_example.py
- Data
  - https://github.com/microsoft/LightGBM/tree/master/examples/regression

In [1]:
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error

In [2]:
print('Loading data...')

df_train = pd.read_csv('data/regression.train', header=None, sep='\t')
df_test = pd.read_csv('data/regression.test', header=None, sep='\t')

Loading data...


In [3]:
df_train.shape

(7000, 29)

In [9]:
df_train.head(3)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,1,0.869,-0.635,0.226,0.327,-0.690,0.754,-0.249,-1.092,0.000,...,-0.010,-0.046,3.102,1.354,0.980,0.978,0.920,0.722,0.989,0.877
1,1,0.908,0.329,0.359,1.498,-0.313,1.096,-0.558,-1.588,2.173,...,-1.139,-0.001,0.000,0.302,0.833,0.986,0.978,0.780,0.992,0.798
2,1,0.799,1.471,-1.636,0.454,0.426,1.105,1.282,1.382,0.000,...,1.129,0.900,0.000,0.910,1.108,0.986,0.951,0.803,0.866,0.780


In [10]:
df_test.shape

(500, 29)

In [11]:
y_train = df_train[0]
y_test = df_test[0]
X_train = df_train.drop(0, axis=1)
X_test = df_test.drop(0, axis=1)

In [12]:
print(type(y_train), type(X_train))

<class 'pandas.core.series.Series'> <class 'pandas.core.frame.DataFrame'>


In [13]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [14]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Starting training...')

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

Starting training...
[1]	valid_0's l1: 0.491812	valid_0's l2: 0.24288
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l1: 0.48798	valid_0's l2: 0.239307
[3]	valid_0's l1: 0.483905	valid_0's l2: 0.235559
[4]	valid_0's l1: 0.478533	valid_0's l2: 0.230771
[5]	valid_0's l1: 0.47333	valid_0's l2: 0.226297
[6]	valid_0's l1: 0.470079	valid_0's l2: 0.223692
[7]	valid_0's l1: 0.466768	valid_0's l2: 0.220941
[8]	valid_0's l1: 0.462971	valid_0's l2: 0.217982
[9]	valid_0's l1: 0.459474	valid_0's l2: 0.215351
[10]	valid_0's l1: 0.45619	valid_0's l2: 0.213064
[11]	valid_0's l1: 0.4532	valid_0's l2: 0.211053
[12]	valid_0's l1: 0.450526	valid_0's l2: 0.209336
[13]	valid_0's l1: 0.447621	valid_0's l2: 0.207492
[14]	valid_0's l1: 0.445178	valid_0's l2: 0.206016
[15]	valid_0's l1: 0.442718	valid_0's l2: 0.204677
[16]	valid_0's l1: 0.440293	valid_0's l2: 0.203224
[17]	valid_0's l1: 0.437015	valid_0's l2: 0.201186
[18]	valid_0's l1: 0.434725	valid_0's l2: 0.199626
[19]	valid_0's l

In [15]:
print('Saving model...')

gbm.save_model('data/simple_example_model.txt')

Saving model...


In [17]:
print('Starting predicting...')

y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

print(y_pred[:10])

Starting predicting...
The rmse of prediction is: 0.44427172467679765
[0.5947354  0.45111956 0.34806607 0.50889704 0.36426725 0.33970103
 0.4413845  0.36607818 0.69598061 0.40820738]
